In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from astropy.modeling import models, fitting

# Configuración de estilo para los gráficos
plt.style.use('default')
sns.set_palette("colorblind")

def analyze_photometry_coherence(df, aperture=3, mag_min=10, mag_max=90):
    """
    Analizar la coherencia entre las magnitudes SPLUS y las del catálogo de Taylor.
    Por defecto usa APERTURE = 3.
    """
    # Definir las correspondencias entre filtros (Taylor filter names deben existir en df)
    filter_correspondences = [
        ('MAG_F378', 'umag', 'F378 vs u-band'),
        ('MAG_F395', 'umag', 'F395 vs u-band (aproximado)'),
        ('MAG_F410', 'gmag', 'F410 vs g-band (aproximado)'),
        ('MAG_F430', 'gmag', 'F430 vs g-band (aproximado)'),
        ('MAG_F515', 'gmag', 'F515 vs g-band'),
        ('MAG_F660', 'rmag', 'F660 vs r-band'),
        ('MAG_F861', 'imag', 'F861 vs i-band')
    ]
    
    results = {}
    n_plots = len(filter_correspondences)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.ravel()
    
    i = -1  # por si acaso no se procesa ningún filtro
    for i, (splus_filter, taylor_filter, title) in enumerate(filter_correspondences):
        splus_col = f'{splus_filter}_{aperture}'
        
        # Verificar que las columnas existen
        if splus_col not in df.columns or taylor_filter not in df.columns:
            print(f"Advertencia: {splus_col} o {taylor_filter} no encontrados. Saltando.")
            continue
            
        # Filtrar datos válidos (excluir valores > mag_max, < mag_min y NaN)
        valid_mask = (
            df[splus_col].notna() &
            df[taylor_filter].notna() &
            np.isfinite(df[splus_col]) &
            np.isfinite(df[taylor_filter]) &
            (df[splus_col] < mag_max) &
            (df[taylor_filter] < mag_max) &
            (df[splus_col] > mag_min) &
            (df[taylor_filter] > mag_min)
        )
        
        splus_mags = df.loc[valid_mask, splus_col]
        taylor_mags = df.loc[valid_mask, taylor_filter]
        
        if len(splus_mags) < 10:
            print(f"No hay suficientes datos válidos para {splus_col} vs {taylor_filter} (N={len(splus_mags)}). Saltando.")
            continue
        
        # Calcular diferencias
        differences = splus_mags - taylor_mags
        
        # Estadísticas
        mean_diff = np.mean(differences)
        median_diff = np.median(differences)
        std_diff = np.std(differences)
        mad_diff = np.median(np.abs(differences - median_diff))  # Desviación absoluta mediana
        correlation = stats.pearsonr(taylor_mags, splus_mags)[0]
        
        # Ajustar una línea recta robusta (LSQ)
        try:
            init_model = models.Linear1D(slope=1, intercept=0)
            fitter = fitting.LinearLSQFitter()
            fitted_model = fitter(init_model, taylor_mags, splus_mags)
            slope, intercept = float(fitted_model.slope.value), float(fitted_model.intercept.value)
        except Exception as e:
            print(f"Warning fitting linear model for {splus_col}: {e}")
            slope, intercept = 1.0, 0.0
        
        results[splus_filter] = {
            'taylor_filter': taylor_filter,
            'slope': slope,
            'intercept': intercept,
            'correlation': correlation,
            'mean_diff': mean_diff,
            'median_diff': median_diff,
            'std_diff': std_diff,
            'mad_diff': mad_diff,
            'n_sources': len(splus_mags),
            'aperture': aperture
        }
        
        # Graficar
        ax = axes[i]
        sc = ax.scatter(taylor_mags, splus_mags, alpha=0.6, s=15, c=differences, 
                       cmap='coolwarm', vmin=np.percentile(differences, 2), vmax=np.percentile(differences, 98))
        
        # Línea de 1:1
        x_min, x_max = min(taylor_mags), max(taylor_mags)
        x_range = np.linspace(x_min, x_max, 200)
        ax.plot(x_range, x_range, 'k--', alpha=0.7, label='1:1')
        
        # Línea ajustada
        ax.plot(x_range, slope*x_range + intercept, 'r-', 
                label=f'y = {slope:.3f}x + {intercept:.3f}')
        
        ax.set_xlabel(f'Taylor {taylor_filter}')
        ax.set_ylabel(f'SPLUS {splus_filter} (aper={aperture})')
        ax.set_title(f'{title}\nr = {correlation:.3f}, Δ_med = {median_diff:.3f} ± MAD {mad_diff:.3f}')
        ax.legend(loc='best', fontsize='small')
        ax.grid(True, alpha=0.3)
        
        # Añadir barra de color para las diferencias (por cada subplot)
        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label('Diferencia (SPLUS - Taylor)')
    
    # Ocultar ejes vacíos (si quedaron)
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    out_png = '../anac_data/splus_taylor_coherence_analysis_aper{}.png'.format(aperture)
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Figura guardada en: {out_png}")
    
    return results

def generate_statistical_summary(results, out_csv='../anac_data/splus_taylor_coherence_summary.csv'):
    """
    Generar un resumen estadístico de la comparación y guardarlo en CSV.
    """
    print("="*60)
    print("ANÁLISIS DE COHERENCIA: FOTOMETRÍA SPLUS vs TAYLOR")
    print("="*60)
    
    summary_data = []
    for splus_filter, stats in results.items():
        summary_data.append({
            'Filtro SPLUS': splus_filter,
            'Filtro Taylor': stats['taylor_filter'],
            'Aperture': stats.get('aperture', ''),
            'N': stats['n_sources'],
            'Correlación': f"{stats['correlation']:.3f}",
            'Pendiente': f"{stats['slope']:.3f}",
            'Intercepto': f"{stats['intercept']:.3f}",
            'Δ media': f"{stats['mean_diff']:.3f}",
            'Δ mediana': f"{stats['median_diff']:.3f}",
            'σ': f"{stats['std_diff']:.3f}",
            'MAD': f"{stats['mad_diff']:.3f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    # Guardar resumen en CSV
    summary_df.to_csv(out_csv, index=False)
    print(f"\nResumen guardado en '{out_csv}'")

# --------------------
# EJECUCIÓN
# --------------------
df = pd.read_csv('../anac_data/Results/all_fields_gc_photometry_buzzo_method.csv')

# Analizar coherencia usando APERTURE = 3
ap = 2
results = analyze_photometry_coherence(df, aperture=ap)

# Generar resumen estadístico
generate_statistical_summary(results, out_csv=f'../anac_data/splus_taylor_coherence_summary_aper{ap}.csv')

# Análisis adicional: Distribución de diferencias por filtro (usando misma apertura)
plt.figure(figsize=(12, 8))
for splus_filter, stats in results.items():
    splus_col = f'{splus_filter}_{ap}'
    taylor_col = stats['taylor_filter']
    valid_mask = (
        df[splus_col].notna() &
        df[taylor_col].notna() &
        np.isfinite(df[splus_col]) &
        np.isfinite(df[taylor_col]) &
        (df[splus_col] < 90) &
        (df[taylor_col] < 90) &
        (df[splus_col] > 10) &
        (df[taylor_col] > 10)
    )
    differences = df.loc[valid_mask, splus_col] - df.loc[valid_mask, taylor_col]
    if len(differences) == 0:
        continue
    plt.hist(differences, bins=30, alpha=0.5, label=f'{splus_filter} - {taylor_col} (N={len(differences)}) mean={np.mean(differences):.3f}')
plt.xlabel('Diferencia (SPLUS - Taylor)')
plt.ylabel('Frecuencia')
plt.title(f'Distribución de diferencias entre fotometría SPLUS y Taylor (aper={ap})')
plt.legend()
plt.grid(True, alpha=0.3)
out_hist = f'../anac_data/splus_taylor_differences_distribution_aper{ap}.png'
plt.savefig(out_hist, dpi=300, bbox_inches='tight')
plt.close()
print(f"Histograma guardado en: {out_hist}")


Figura guardada en: ../anac_data/splus_taylor_coherence_analysis_aper2.png
ANÁLISIS DE COHERENCIA: FOTOMETRÍA SPLUS vs TAYLOR
Filtro SPLUS Filtro Taylor  Aperture   N Correlación Pendiente Intercepto Δ media Δ mediana     σ   MAD
    MAG_F378          umag         2 106       0.792     0.525      4.630  -5.984    -5.946 0.594 0.433
    MAG_F395          umag         2 111       0.562     0.360     13.236  -1.110    -1.038 0.805 0.610
    MAG_F410          gmag         2 136       0.735     0.719      6.608   0.766     0.821 0.612 0.401
    MAG_F430          gmag         2 148       0.821     0.739      6.332   0.893     0.944 0.490 0.311
    MAG_F515          gmag         2 161       0.909     0.930      2.302   0.842     0.805 0.364 0.195
    MAG_F660          rmag         2 169       0.756     0.943      1.409   0.261     0.274 0.635 0.109
    MAG_F861          imag         2 168       0.882     0.996     -0.193  -0.278    -0.357 0.403 0.190

Resumen guardado en '../anac_data/splus_t